# 1. 크롤링과 함께 쓰는 mongoDB 예제

### cine21 인물 랭킹 알아내기
  - http://www.cine21.com/rank/person/
  - 조회를 눌렀을 때, Go to Network -> content 의 Request URL/Method 와 Form 데이터 알아내기
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form Data
      - section:actor
      - period_start:2021-10
      - gender:all
      - page:1
    - 하단부 페이지를 누를 때마다, Form Data 의 page 값이 바뀜

In [1]:
import requests as req
import re
import datetime
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pymongo

In [2]:
# 몽고DB 서버 연결
username = 'rapa00'
password = '1234'
conn = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))

In [3]:
# db, collection 연결(없으면 생성)
# python과 연동되었을 때, db와 collection의 경우, document insert 싯점에 생성됨
mongodb = conn.cine21
actor_collection = mongodb.actor_collection # collection 객체 생성자


In [24]:
actor_list = actor_collection.find()

In [35]:
# actor 정보가 있는지 확인하기
# 데이터가 있다면 정보를 가져오고 데이터가 없는상태에서는 에러가 뜬다
actor_list = actor_collection.find()
for actor in actor_list:
    print(actor['actor'])

박정민
이성민
임윤아
노회찬
이수경
박영남
김환진


In [5]:
# request url
cine21_url = 'http://www.cine21.com/rank/person/content'
month = "2021-10"
year = "2020-11"

# post 방식으로 데이터를 가져오기 위한 request 시 필요 정보
conditions = dict()
conditions['section'] = 'actor'
conditions['period_start'] = month
conditions['gender'] = 'all'
conditions['page'] = 1

# post 방식으로 request 요청
# url은 get방식과 post방식이 있고 get 방식은 보안상 위험이 크다
response = req.post(cine21_url, data = conditions)

In [6]:
response

<Response [200]>

In [7]:

# soup  = bs(response.content.decode('utf-8'), 'html.parser')
soup  = bs(response.content, 'html.parser')
soup

 <ul class="people_list">
<li class="people_li">
<a href="/db/person/info/?person_id=74826"><img alt="" class="people_thumb" src="https://image.cine21.com/resize/cine21/person/2019/0828/11_54_52__5d65ecfc13ca6[X145,145].jpg" target="_blank"/></a>
<div class="name"><a href="/db/person/info/?person_id=74826">박정민(1편)</a></div>
<ul class="num_info">
<li><span class="tit">흥행지수</span><strong>161,725</strong></li>
<!--
						<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>
						-->
</ul>
<!-- 영화포스터는 최대 5개까지만 -->
<ul class="mov_list">
<li>
<a href="/movie/info/?movie_id=57227">
<img alt="" class="thumb" src="https://image.cine21.com/resize/cine21/poster/2021/0928/10_31_26__6152706e344e6[X85,120].jpg" target="_blank"/>
<span>기적</span>
</a>
</li>
</ul>
<!-- 순위 --><span class="grade">1</span>
</li>
<li class="people_li">
<a href="/db/person/info/?person_id=25123"><img alt="" class="people_thumb" src="https://image.cine21.com/resize/cine21/person/

In [ ]:
#rank_holder > ul > li:nth-child(1) > div > a

In [8]:
tds = soup.select("div.name")
for i in tds:
    i = tds[0].get_text(strip=True)
print(i)

박정민(1편)


In [9]:
# 정규 표현식 참고 : https://regexr.com/
# 문자, 숫자를 찾음 : \w
# 특수 기호 () 문자로 인식 : \( \)

# 2. 반복 표현 ?, *, +
# + : 앞 문자가 1번 또는 그 이상 반복되는 패턴
# ? : 앞 문자가 0번 또는 1번 표시되는 패턴(없어도 되고, 한번 있어도 되는 패턴)
# * : 앞 문자가 0번 또는 그이상 반복되는 패턴
import re
test_data = '마동석(17편)'
#re.sub('정규표현식', '변경데이터', test_data)
print(re.sub("\(\w+\)", "", test_data))

마동석


In [10]:
# [실습]
# 한 페이지 내의 배우이름만 가져오기
import re

name_list = []
for tt in tds:
    print(re.sub("\(\w+\)", "", tt.getText(strip=True)))
    name_list.append((re.sub("\(\w+\)", "", tt.getText(strip=True))))


박정민
이성민
임윤아
노회찬
이수경
박영남
김환진


### 각 배우별 상세 정보를 document에 넣고 싶다.
* 각 배우별 상세 정보를 별도 컬럼으로 만들려했더니, 각 배우별 상세 정보 항목이 다르다!
* 모든 상세 정보 항목을 컬럼으로 만들고, 각 컬럼에 매칭되는 컬럼값을 넣기가 쉽지 않다. 코드도 복잡하고!
* Mongodb는 NoSQL -> 통째로 집어넣자.!

* embedded document
  - document 의 컬럼값으로 document를 넣을 수 있다.

In [ ]:
#info_content > div.default_info_area > ul.default_info
# http://www.cine21.com/inc/www/css/content1.css
# http://www.cine21.com/db/person/info/?person_id=74826
# http://www.cine21.com/db/person/info/?person_id=25123

In [11]:
data_li= []
for i in tds:
    li = " http://www.cine21.com" + i.select_one('a').attrs['href']
    res_url = req.get(li)
    soup_actor = bs(res_url.content, "html.parser")
    default_info = soup_actor.select_one('ul.default_info')
    print(re.sub("\(\w+\)", "", default_info.getText(strip=True)))
   


직업배우생년월일1987-02-25성별남신장/체중178cm, 63kg학교한국예술종합학교 영상원 연극원 연기과
직업배우생년월일1968-10-15성별남신장/체중178cm
다른 이름소녀시대; girlsgeneration; girls generation직업가수생년월일1990-05-30성별여홈페이지https://www.instagram.com/yoona__lim/소속사SM엔터테인먼트
생년월일1956-08-31사망2018-07-23성별남홈페이지https://twitter.com/hcrohhttps://www.facebook.com/omyChans
직업배우생년월일1996-10-24성별여
직업성우생년월일1946-10-08성별여
직업성우생년월일1952-10-03성별남


In [12]:
tds[0].select_one('a').attrs['href']

'/db/person/info/?person_id=74826'

In [31]:
actor_real=list()
for i in tds:
    li = " http://www.cine21.com" + i.select_one('a').attrs['href']
    res_url = req.get(li)
    soup_actor = bs(res_url.content, "html.parser")
    default_info = soup_actor.select_one('ul.default_info')   
    
    actor_details = default_info.select('li')
    actor_info_dict = dict()
    
    for actor_detail in actor_details:
        actor_detail_key = actor_detail.select_one('span.tit').get_text(strip=True)
        actor_detail_value = re.sub('<span.*?>.*?</span>','',str(actor_detail))
        # print(actor_detail_value)
        actor_detail_value = re.sub('<.*?>','',actor_detail_value).strip()
        # print(actor_detail_value)


        # a = actor_detail
        # print(type(a))
        # b = actor_detail.select_one('span').extract() #.get_text()
        # print(type(b))
        # c = actor_detail #.get_text()
        # print(type(c))
        # print("-" * 10)

        actor_info_dict[actor_detail_key] = actor_detail_value
    actor_real.append(actor_info_dict)
       
print(actor_real)

[{'직업': '배우', '생년월일': '1987-02-25', '성별': '남', '신장/체중': '178cm, 63kg', '학교': '한국예술종합학교 영상원 연극원 연기과'}, {'직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm'}, {'다른 이름': '소녀시대; girlsgeneration; girls generation', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': 'https://www.instagram.com/yoona__lim/', '소속사': 'SM엔터테인먼트'}, {'생년월일': '1956-08-31', '사망': '2018-07-23', '성별': '남', '홈페이지': 'https://twitter.com/hcroh\nhttps://www.facebook.com/omyChans'}, {'직업': '배우', '생년월일': '1996-10-24', '성별': '여'}, {'직업': '성우', '생년월일': '1946-10-08', '성별': '여'}, {'직업': '성우', '생년월일': '1952-10-03', '성별': '남'}]


### 각 배우별 출연 영화를 document에 저장하고 싶다.
  - 출연 영화는 한 개가 될 수도 있고, 여러 개가 될 수도 있음
  - 파이썬은 리스트, mongodb document는 컬럼에 배열(array)로 넣으면 됨 

In [16]:
movie_list = list()
movies = soup.select('li.people_li ul.mov_list')
# print(movies)

for movie in movies:
    actor_movies = list()
    movie_titles = movie.select('ul.mov_list li a span')
    for movie_title in movie_titles:
        actor_movies.append(movie_title.text)
        print(movie_title.text)
    movie_list.append(actor_movies)
    print('-'*10)

기적
----------
기적
----------
기적
----------
노회찬6411
----------
기적
----------
극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들
----------
극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들
----------


In [19]:
tdds = soup.select("ul.num_info strong")
for i in tdds:
    i = tdds[0].get_text(strip=True)


num_list = []
for ttt in tdds:
    print(re.sub("\(\w+\)", "", ttt.getText(strip=True)))
    num_list.append((re.sub("\(\w+\)", "", ttt.getText(strip=True))))

161,725
121,294
80,862
51,594
40,431
27,908
24,419


In [ ]:
#rank_holder > ul > li:nth-child(1) > ul.mov_list > li > a > span

In [32]:
print(name_list)
print(actor_real)
print(num_list)
print(movie_list)


['박정민', '이성민', '임윤아', '노회찬', '이수경', '박영남', '김환진']
[{'직업': '배우', '생년월일': '1987-02-25', '성별': '남', '신장/체중': '178cm, 63kg', '학교': '한국예술종합학교 영상원 연극원 연기과'}, {'직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm'}, {'다른 이름': '소녀시대; girlsgeneration; girls generation', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': 'https://www.instagram.com/yoona__lim/', '소속사': 'SM엔터테인먼트'}, {'생년월일': '1956-08-31', '사망': '2018-07-23', '성별': '남', '홈페이지': 'https://twitter.com/hcroh\nhttps://www.facebook.com/omyChans'}, {'직업': '배우', '생년월일': '1996-10-24', '성별': '여'}, {'직업': '성우', '생년월일': '1946-10-08', '성별': '여'}, {'직업': '성우', '생년월일': '1952-10-03', '성별': '남'}]
['161,725', '121,294', '80,862', '51,594', '40,431', '27,908', '24,419']
[['기적'], ['기적'], ['기적'], ['노회찬6411'], ['기적'], ['극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들'], ['극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들']]


### insert_one() 로 하나씩 데이터 입력하기 (반복문과 함께 사용하면, 여러 데이터를 넣을 수 있음)

- actor_list: 배우 이름
- actor_details: 배우 상세 정보
- actor_rate: 흥행 지수
- date: 기준월
- movie_list: 출연 영화 리스트!

In [34]:

for num, actor in enumerate(name_list):
    actor_collection.insert_one({
        "actor":name_list[num],
        "actor_details": actor_real[num],
        "actor_hit":num_list[num],
        "date":month,
        "movie_list":movie_list[num]
        } )
print(actor_collection)


Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'cine21'), 'actor_collection')


In [ ]:
# actors_info = dict()
# for num, actor in enumerate(actor_list):
#     actors_info(
#         {"actor":actor,
#         "actor_details": actor_info_dict[num],
#         "actor_hit":data_list1[num],
#         "date":month,
#         "movie_list":movie_list[num]
#         } )
# print(actors_info)
# actor_collection.insert_many(actors_info)

{}


TypeError: documents must be a non-empty list

In [ ]:
# actor_collection.update_many({},{"$rename":{"actor":"actor_name"}})

### collection 삭제하기

In [ ]:
# collection drop
actor_collection.drop()

In [36]:
# collection 찾기
docs = actor_collection.find()
docs

In [37]:
for doc in docs:
    print(doc)

{'_id': ObjectId('61847abb045c4b7daa2c38b0'), 'actor': '박정민', 'actor_details': {'직업': '배우', '생년월일': '1987-02-25', '성별': '남', '신장/체중': '178cm, 63kg', '학교': '한국예술종합학교 영상원 연극원 연기과'}, 'actor_hit': '161,725', 'date': '2021-10', 'movie_list': ['기적']}
{'_id': ObjectId('61847abb045c4b7daa2c38b1'), 'actor': '이성민', 'actor_details': {'직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm'}, 'actor_hit': '121,294', 'date': '2021-10', 'movie_list': ['기적']}
{'_id': ObjectId('61847abb045c4b7daa2c38b2'), 'actor': '임윤아', 'actor_details': {'다른 이름': '소녀시대; girlsgeneration; girls generation', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': 'https://www.instagram.com/yoona__lim/', '소속사': 'SM엔터테인먼트'}, 'actor_hit': '80,862', 'date': '2021-10', 'movie_list': ['기적']}
{'_id': ObjectId('61847abb045c4b7daa2c38b3'), 'actor': '노회찬', 'actor_details': {'생년월일': '1956-08-31', '사망': '2018-07-23', '성별': '남', '홈페이지': 'https://twitter.com/hcroh\nhttps://www.facebook.com/omyChans'}, 'actor_hit': '51,594', 'date':

### Dictionary 타입으로 만들어서 한번에 insert_many() 로 데이터 입력하기

### Update (컬럼명 변경 예제)

### 컬렉션 객체 이름도 바꿀 수 있겠지요
actor_collection -> actors_info 로 변경